In [1]:
import glob
import os
import sys

import pandas as pd
import cv2

from PIL import Image
import mediapipe as mp
import numpy as np

In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection

In [3]:
path_image = './test_images_self_created/'

f = []
folders = glob.glob(os.path.join(path_image, '*'))

In [49]:
for f in folders:
    name_pure = os.path.split(f)[1]
    files = glob.glob(os.path.join(f, '*'))
    for file in files:
        if not 'scene' in os.path.split(file)[1]:
            os.remove(file)

In [6]:
for f in folders:
    name_pure = os.path.split(f)[1]
    files = glob.glob(os.path.join(f, '*'))
    for file in files:
        try:
            file_name = os.path.split(file)[1].split('scene')[0]
            with Image.open(file) as image:
                    image = np.array(image)
                    ih, iw, ic = image.shape

                    with mp_face_mesh.FaceMesh( static_image_mode=True, max_num_faces=2, min_detection_confidence=0.5) as face_mesh:
                        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
                        face_detected = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    if not results.multi_face_landmarks or not face_detected.detections:
                        continue


                    face_point = face_detected.detections[0].location_data.relative_bounding_box  # ['relative_bounding_box']
                    # Extracting the coordinates for face bounding box
                    x = int(face_point.xmin * iw) - 30
                    y = int(face_point.ymin * ih) - 100
                    w = int(face_point.width * iw) + 60
                    h = int(face_point.height * ih) + 100
                    x2 = x + w 
                    y2 = y + h

                    image = Image.fromarray(image)

                    region=image.crop([x,y,x2,y2])
                    region = region.resize((300,300))

                    region.save(file)
        except:
            print(f)
            print(file)
            continue
